In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(color_codes=True)

## Load the data

In [2]:
df = pd.read_csv('../cleaned_data/ISAT.csv', index_col='Date', parse_dates=True)
df.head()

7 DAYS MA  14 DAYS MA  21 DAYS MA  7 DAYS STD DEV     RSI 7  \
Date                                                                      
2012-10-17   0.437800    0.340388    0.341359        0.526150  0.750237   
2012-10-18   0.460510    0.358130    0.342201       -0.036486  0.560174   
2012-10-19   0.465556    0.374604    0.342201       -0.021718  0.560174   
2012-10-22   0.463033    0.394880    0.348093       -0.106151  0.884145   
2012-10-23   0.473126    0.415157    0.362404        0.241153  0.659316   

              RSI 14    RSI 21     MFI 7    MFI 14    MFI 21  dayofweek  \
Date                                                                      
2012-10-17  0.216300  0.113942  0.235652  0.709952  0.010976        0.0   
2012-10-18  0.121030  0.039230  0.163048  0.609002  0.051422        0.5   
2012-10-19  0.121030  0.039230 -0.331469  0.646230  0.158623        1.0   
2012-10-22  0.391605  0.265115  0.014964  0.619132  0.610507       -1.0   
2012-10-23  0.282592  0.182832  0.236528  0.608329  0.870434       -0.5   

            quarter  month  year  dayofyear  dayofmonth  weekofyear     Close  
Date                                                                           
2012-10-17      0.5    0.5  -1.0   0.594005    0.066667    0.592593  0.482270  
2012-10-18      0.5    0.5  -1.0   0.599455    0.133333    0.592593  0.464539  
2012-10-19      0.5    0.5  -1.0   0.604905    0.200000    0.592593  0.464539  
2012-10-22      0.5    0.5  -1.0   0.621253    0.400000    0.629630  0.517730  
2012-10-23      0.5    0.5  -1.0   0.626703    0.466667    0.629630  0.500000

### Plot the Close price - our target

In [3]:
df['Close'].plot(figsize=(18,4))
plt.legend(['Close Price'])
plt.title('ISAT.JK Close Price')
plt.show()

### Split the data
- Split the data into independent variables and target variable
- Split the data into train and test set (80:20)

In [4]:
train_len = int(len(df)*.8)
train_dataset = df[:train_len]
test_dataset = df[train_len:len(df)]

In [5]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Close')
test_labels = test_features.pop('Close')

## Modeling

### Grid Search to find the best combination of hyperparameters

In [6]:
# ================
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 
#=========================================================================
regressor = xgb.XGBRegressor(eval_metric='rmse')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"max_depth":    [3, 4, 5],
              "n_estimators": [500, 600, 700, 1000],
              "learning_rate": [0.01, 0.015],
              "base_score": [0.5],
              "booster": ["gbtree"],
              "early_stopping_rounds": [50]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5).fit(train_features, train_labels, eval_set=[(train_features, train_labels)])

[0]	validation_0-rmse:0.80644
[1]	validation_0-rmse:0.79848
[2]	validation_0-rmse:0.79060
[3]	validation_0-rmse:0.78280
[4]	validation_0-rmse:0.77508
[5]	validation_0-rmse:0.76744
[6]	validation_0-rmse:0.75987
[7]	validation_0-rmse:0.75238
[8]	validation_0-rmse:0.74496
[9]	validation_0-rmse:0.73762
[10]	validation_0-rmse:0.73036
[11]	validation_0-rmse:0.72317
[12]	validation_0-rmse:0.71604
[13]	validation_0-rmse:0.70900
[14]	validation_0-rmse:0.70201
[15]	validation_0-rmse:0.69510
[16]	validation_0-rmse:0.68827
[17]	validation_0-rmse:0.68151
[18]	validation_0-rmse:0.67481
[19]	validation_0-rmse:0.66818
[20]	validation_0-rmse:0.66161
[21]	validation_0-rmse:0.65511
[22]	validation_0-rmse:0.64867
[23]	validation_0-rmse:0.64230
[24]	validation_0-rmse:0.63599
[25]	validation_0-rmse:0.62975
[26]	validation_0-rmse:0.62357
[27]	validation_0-rmse:0.61745
[28]	validation_0-rmse:0.61139
[29]	validation_0-rmse:0.60539
[30]	validation_0-rmse:0.59947
[31]	validation_0-rmse:0.59360
[32]	validation_0-

[260]	validation_0-rmse:0.07481
[261]	validation_0-rmse:0.07427
[262]	validation_0-rmse:0.07374
[263]	validation_0-rmse:0.07321
[264]	validation_0-rmse:0.07268
[265]	validation_0-rmse:0.07216
[266]	validation_0-rmse:0.07165
[267]	validation_0-rmse:0.07115
[268]	validation_0-rmse:0.07064
[269]	validation_0-rmse:0.07015
[270]	validation_0-rmse:0.06966
[271]	validation_0-rmse:0.06918
[272]	validation_0-rmse:0.06871
[273]	validation_0-rmse:0.06824
[274]	validation_0-rmse:0.06776
[275]	validation_0-rmse:0.06730
[276]	validation_0-rmse:0.06685
[277]	validation_0-rmse:0.06640
[278]	validation_0-rmse:0.06595
[279]	validation_0-rmse:0.06550
[280]	validation_0-rmse:0.06507
[281]	validation_0-rmse:0.06463
[282]	validation_0-rmse:0.06421
[283]	validation_0-rmse:0.06379
[284]	validation_0-rmse:0.06336
[285]	validation_0-rmse:0.06296
[286]	validation_0-rmse:0.06255
[287]	validation_0-rmse:0.06215
[288]	validation_0-rmse:0.06176
[289]	validation_0-rmse:0.06136
[290]	validation_0-rmse:0.06097
[291]	va

[17]	validation_0-rmse:0.68125
[18]	validation_0-rmse:0.67452
[19]	validation_0-rmse:0.66787
[20]	validation_0-rmse:0.66130
[21]	validation_0-rmse:0.65477
[22]	validation_0-rmse:0.64833
[23]	validation_0-rmse:0.64202
[24]	validation_0-rmse:0.63568
[25]	validation_0-rmse:0.62943
[26]	validation_0-rmse:0.62325
[27]	validation_0-rmse:0.61710
[28]	validation_0-rmse:0.61103
[29]	validation_0-rmse:0.60500
[30]	validation_0-rmse:0.59912
[31]	validation_0-rmse:0.59330
[32]	validation_0-rmse:0.58745
[33]	validation_0-rmse:0.58168
[34]	validation_0-rmse:0.57597
[35]	validation_0-rmse:0.57030
[36]	validation_0-rmse:0.56477
[37]	validation_0-rmse:0.55923
[38]	validation_0-rmse:0.55381
[39]	validation_0-rmse:0.54836
[40]	validation_0-rmse:0.54298
[41]	validation_0-rmse:0.53764
[42]	validation_0-rmse:0.53238
[43]	validation_0-rmse:0.52718
[44]	validation_0-rmse:0.52200
[45]	validation_0-rmse:0.51694
[46]	validation_0-rmse:0.51187
[47]	validation_0-rmse:0.50687
[48]	validation_0-rmse:0.50189
[49]	val

[276]	validation_0-rmse:0.06464
[277]	validation_0-rmse:0.06417
[278]	validation_0-rmse:0.06372
[279]	validation_0-rmse:0.06327
[280]	validation_0-rmse:0.06284
[281]	validation_0-rmse:0.06240
[282]	validation_0-rmse:0.06195
[283]	validation_0-rmse:0.06153
[284]	validation_0-rmse:0.06110
[285]	validation_0-rmse:0.06068
[286]	validation_0-rmse:0.06026
[287]	validation_0-rmse:0.05986
[288]	validation_0-rmse:0.05946
[289]	validation_0-rmse:0.05906
[290]	validation_0-rmse:0.05866
[291]	validation_0-rmse:0.05828
[292]	validation_0-rmse:0.05790
[293]	validation_0-rmse:0.05752
[294]	validation_0-rmse:0.05715
[295]	validation_0-rmse:0.05677
[296]	validation_0-rmse:0.05640
[297]	validation_0-rmse:0.05603
[298]	validation_0-rmse:0.05568
[299]	validation_0-rmse:0.05531
[300]	validation_0-rmse:0.05497
[301]	validation_0-rmse:0.05462
[302]	validation_0-rmse:0.05428
[303]	validation_0-rmse:0.05394
[304]	validation_0-rmse:0.05361
[305]	validation_0-rmse:0.05327
[306]	validation_0-rmse:0.05295
[307]	va

[34]	validation_0-rmse:0.57730
[35]	validation_0-rmse:0.57169
[36]	validation_0-rmse:0.56613
[37]	validation_0-rmse:0.56061
[38]	validation_0-rmse:0.55516
[39]	validation_0-rmse:0.54976
[40]	validation_0-rmse:0.54443
[41]	validation_0-rmse:0.53915
[42]	validation_0-rmse:0.53390
[43]	validation_0-rmse:0.52873
[44]	validation_0-rmse:0.52359
[45]	validation_0-rmse:0.51851
[46]	validation_0-rmse:0.51348
[47]	validation_0-rmse:0.50851
[48]	validation_0-rmse:0.50358
[49]	validation_0-rmse:0.49870
[50]	validation_0-rmse:0.49386
[51]	validation_0-rmse:0.48908
[52]	validation_0-rmse:0.48436
[53]	validation_0-rmse:0.47967
[54]	validation_0-rmse:0.47502
[55]	validation_0-rmse:0.47043
[56]	validation_0-rmse:0.46588
[57]	validation_0-rmse:0.46138
[58]	validation_0-rmse:0.45691
[59]	validation_0-rmse:0.45249
[60]	validation_0-rmse:0.44813
[61]	validation_0-rmse:0.44380
[62]	validation_0-rmse:0.43951
[63]	validation_0-rmse:0.43528
[64]	validation_0-rmse:0.43108
[65]	validation_0-rmse:0.42695
[66]	val

...

[888]	validation_0-rmse:0.02522
[889]	validation_0-rmse:0.02521
[890]	validation_0-rmse:0.02521
[891]	validation_0-rmse:0.02521
[892]	validation_0-rmse:0.02521
[893]	validation_0-rmse:0.02520
[894]	validation_0-rmse:0.02520
[895]	validation_0-rmse:0.02520
[896]	validation_0-rmse:0.02520
[897]	validation_0-rmse:0.02520
[898]	validation_0-rmse:0.02520
[899]	validation_0-rmse:0.02519
[900]	validation_0-rmse:0.02519
[901]	validation_0-rmse:0.02519
[902]	validation_0-rmse:0.02519
[903]	validation_0-rmse:0.02519
[904]	validation_0-rmse:0.02519
[905]	validation_0-rmse:0.02518
[906]	validation_0-rmse:0.02518
[907]	validation_0-rmse:0.02518
[908]	validation_0-rmse:0.02518
[909]	validation_0-rmse:0.02517
[910]	validation_0-rmse:0.02517
[911]	validation_0-rmse:0.02517
[912]	validation_0-rmse:0.02517
[913]	validation_0-rmse:0.02517
[914]	validation_0-rmse:0.02517
[915]	validation_0-rmse:0.02517
[916]	validation_0-rmse:0.02516
[917]	validation_0-rmse:0.02517
[918]	validation_0-rmse:0.02516
[919]	va

[148]	validation_0-rmse:0.09998
[149]	validation_0-rmse:0.09875
[150]	validation_0-rmse:0.09754
[151]	validation_0-rmse:0.09634
[152]	validation_0-rmse:0.09517
[153]	validation_0-rmse:0.09402
[154]	validation_0-rmse:0.09288
[155]	validation_0-rmse:0.09176
[156]	validation_0-rmse:0.09066
[157]	validation_0-rmse:0.08958
[158]	validation_0-rmse:0.08852
[159]	validation_0-rmse:0.08746
[160]	validation_0-rmse:0.08642
[161]	validation_0-rmse:0.08541
[162]	validation_0-rmse:0.08441
[163]	validation_0-rmse:0.08342
[164]	validation_0-rmse:0.08244
[165]	validation_0-rmse:0.08149
[166]	validation_0-rmse:0.08054
[167]	validation_0-rmse:0.07962
[168]	validation_0-rmse:0.07871
[169]	validation_0-rmse:0.07781
[170]	validation_0-rmse:0.07693
[171]	validation_0-rmse:0.07606
[172]	validation_0-rmse:0.07521
[173]	validation_0-rmse:0.07437
[174]	validation_0-rmse:0.07354
[175]	validation_0-rmse:0.07273
[176]	validation_0-rmse:0.07192
[177]	validation_0-rmse:0.07113
[178]	validation_0-rmse:0.07036
[179]	va

[405]	validation_0-rmse:0.02324
[406]	validation_0-rmse:0.02320
[407]	validation_0-rmse:0.02318
[408]	validation_0-rmse:0.02314
[409]	validation_0-rmse:0.02311
[410]	validation_0-rmse:0.02308
[411]	validation_0-rmse:0.02305
[412]	validation_0-rmse:0.02301
[413]	validation_0-rmse:0.02297
[414]	validation_0-rmse:0.02294
[415]	validation_0-rmse:0.02292
[416]	validation_0-rmse:0.02288
[417]	validation_0-rmse:0.02283
[418]	validation_0-rmse:0.02281
[419]	validation_0-rmse:0.02277
[420]	validation_0-rmse:0.02275
[421]	validation_0-rmse:0.02272
[422]	validation_0-rmse:0.02269
[423]	validation_0-rmse:0.02266
[424]	validation_0-rmse:0.02262
[425]	validation_0-rmse:0.02260
[426]	validation_0-rmse:0.02258
[427]	validation_0-rmse:0.02254
[428]	validation_0-rmse:0.02251
[429]	validation_0-rmse:0.02248
[430]	validation_0-rmse:0.02245
[431]	validation_0-rmse:0.02242
[432]	validation_0-rmse:0.02239
[433]	validation_0-rmse:0.02236
[434]	validation_0-rmse:0.02234
[435]	validation_0-rmse:0.02232
[436]	va

[662]	validation_0-rmse:0.01888
[663]	validation_0-rmse:0.01886
[664]	validation_0-rmse:0.01886
[665]	validation_0-rmse:0.01886
[666]	validation_0-rmse:0.01884
[667]	validation_0-rmse:0.01883
[668]	validation_0-rmse:0.01881
[669]	validation_0-rmse:0.01880
[670]	validation_0-rmse:0.01879
[671]	validation_0-rmse:0.01879
[672]	validation_0-rmse:0.01878
[673]	validation_0-rmse:0.01878
[674]	validation_0-rmse:0.01876
[675]	validation_0-rmse:0.01875
[676]	validation_0-rmse:0.01874
[677]	validation_0-rmse:0.01874
[678]	validation_0-rmse:0.01872
[679]	validation_0-rmse:0.01870
[680]	validation_0-rmse:0.01869
[681]	validation_0-rmse:0.01868
[682]	validation_0-rmse:0.01867
[683]	validation_0-rmse:0.01866
[684]	validation_0-rmse:0.01865
[685]	validation_0-rmse:0.01864
[686]	validation_0-rmse:0.01863
[687]	validation_0-rmse:0.01862
[688]	validation_0-rmse:0.01861
[689]	validation_0-rmse:0.01860
[690]	validation_0-rmse:0.01858
[691]	validation_0-rmse:0.01857
[692]	validation_0-rmse:0.01857
[693]	va

[919]	validation_0-rmse:0.01657
[920]	validation_0-rmse:0.01657
[921]	validation_0-rmse:0.01656
[922]	validation_0-rmse:0.01655
[923]	validation_0-rmse:0.01654
[924]	validation_0-rmse:0.01654
[925]	validation_0-rmse:0.01653
[926]	validation_0-rmse:0.01651
[927]	validation_0-rmse:0.01650
[928]	validation_0-rmse:0.01650
[929]	validation_0-rmse:0.01649
[930]	validation_0-rmse:0.01648
[931]	validation_0-rmse:0.01648
[932]	validation_0-rmse:0.01647
[933]	validation_0-rmse:0.01646
[934]	validation_0-rmse:0.01645
[935]	validation_0-rmse:0.01645
[936]	validation_0-rmse:0.01644
[937]	validation_0-rmse:0.01643
[938]	validation_0-rmse:0.01641
[939]	validation_0-rmse:0.01641
[940]	validation_0-rmse:0.01640
[941]	validation_0-rmse:0.01640
[942]	validation_0-rmse:0.01640
[943]	validation_0-rmse:0.01639
[944]	validation_0-rmse:0.01638
[945]	validation_0-rmse:0.01638
[946]	validation_0-rmse:0.01637
[947]	validation_0-rmse:0.01636
[948]	validation_0-rmse:0.01635
[949]	validation_0-rmse:0.01634
[950]	va

In [7]:
print("The best hyperparameters are ",search.best_params_)

The best hyperparameters are  {'base_score': 0.5, 'booster': 'gbtree', 'early_stopping_rounds': 50, 'learning_rate': 0.015, 'max_depth': 3, 'n_estimators': 1000}


In [8]:
model = xgb.XGBRegressor(learning_rate=search.best_params_['learning_rate'],
                           n_estimators=search.best_params_['n_estimators'],
                           max_depth=search.best_params_['max_depth'],
                           base_score=search.best_params_['base_score'],
                           booster=search.best_params_['booster'],
                           early_stopping_rounds=search.best_params_['early_stopping_rounds'])

model = model.fit(train_features, train_labels, eval_set=[(train_features, train_labels)])

[0]	validation_0-rmse:0.80245
[1]	validation_0-rmse:0.79060
[2]	validation_0-rmse:0.77894
[3]	validation_0-rmse:0.76744
[4]	validation_0-rmse:0.75612
[5]	validation_0-rmse:0.74498
[6]	validation_0-rmse:0.73399
[7]	validation_0-rmse:0.72318
[8]	validation_0-rmse:0.71253
[9]	validation_0-rmse:0.70204
[10]	validation_0-rmse:0.69171
[11]	validation_0-rmse:0.68152
[12]	validation_0-rmse:0.67149
[13]	validation_0-rmse:0.66162
[14]	validation_0-rmse:0.65189
[15]	validation_0-rmse:0.64231
[16]	validation_0-rmse:0.63287
[17]	validation_0-rmse:0.62358
[18]	validation_0-rmse:0.61442
[19]	validation_0-rmse:0.60540
[20]	validation_0-rmse:0.59652
[21]	validation_0-rmse:0.58777
[22]	validation_0-rmse:0.57915
[23]	validation_0-rmse:0.57066
[24]	validation_0-rmse:0.56230
[25]	validation_0-rmse:0.55407
[26]	validation_0-rmse:0.54595
[27]	validation_0-rmse:0.53796
[28]	validation_0-rmse:0.53009
[29]	validation_0-rmse:0.52235
[30]	validation_0-rmse:0.51471
[31]	validation_0-rmse:0.50719
[32]	validation_0-

[260]	validation_0-rmse:0.03573
[261]	validation_0-rmse:0.03554
[262]	validation_0-rmse:0.03533
[263]	validation_0-rmse:0.03514
[264]	validation_0-rmse:0.03494
[265]	validation_0-rmse:0.03475
[266]	validation_0-rmse:0.03456
[267]	validation_0-rmse:0.03438
[268]	validation_0-rmse:0.03420
[269]	validation_0-rmse:0.03402
[270]	validation_0-rmse:0.03384
[271]	validation_0-rmse:0.03366
[272]	validation_0-rmse:0.03348
[273]	validation_0-rmse:0.03331
[274]	validation_0-rmse:0.03314
[275]	validation_0-rmse:0.03299
[276]	validation_0-rmse:0.03283
[277]	validation_0-rmse:0.03268
[278]	validation_0-rmse:0.03252
[279]	validation_0-rmse:0.03238
[280]	validation_0-rmse:0.03222
[281]	validation_0-rmse:0.03207
[282]	validation_0-rmse:0.03193
[283]	validation_0-rmse:0.03178
[284]	validation_0-rmse:0.03164
[285]	validation_0-rmse:0.03152
[286]	validation_0-rmse:0.03139
[287]	validation_0-rmse:0.03125
[288]	validation_0-rmse:0.03112
[289]	validation_0-rmse:0.03098
[290]	validation_0-rmse:0.03087
[291]	va

[517]	validation_0-rmse:0.02069
[518]	validation_0-rmse:0.02068
[519]	validation_0-rmse:0.02067
[520]	validation_0-rmse:0.02066
[521]	validation_0-rmse:0.02064
[522]	validation_0-rmse:0.02063
[523]	validation_0-rmse:0.02062
[524]	validation_0-rmse:0.02061
[525]	validation_0-rmse:0.02059
[526]	validation_0-rmse:0.02058
[527]	validation_0-rmse:0.02057
[528]	validation_0-rmse:0.02056
[529]	validation_0-rmse:0.02055
[530]	validation_0-rmse:0.02054
[531]	validation_0-rmse:0.02052
[532]	validation_0-rmse:0.02051
[533]	validation_0-rmse:0.02050
[534]	validation_0-rmse:0.02049
[535]	validation_0-rmse:0.02047
[536]	validation_0-rmse:0.02046
[537]	validation_0-rmse:0.02045
[538]	validation_0-rmse:0.02043
[539]	validation_0-rmse:0.02042
[540]	validation_0-rmse:0.02040
[541]	validation_0-rmse:0.02039
[542]	validation_0-rmse:0.02038
[543]	validation_0-rmse:0.02037
[544]	validation_0-rmse:0.02036
[545]	validation_0-rmse:0.02035
[546]	validation_0-rmse:0.02034
[547]	validation_0-rmse:0.02033
[548]	va

[773]	validation_0-rmse:0.01782
[774]	validation_0-rmse:0.01781
[775]	validation_0-rmse:0.01780
[776]	validation_0-rmse:0.01778
[777]	validation_0-rmse:0.01777
[778]	validation_0-rmse:0.01776
[779]	validation_0-rmse:0.01776
[780]	validation_0-rmse:0.01775
[781]	validation_0-rmse:0.01774
[782]	validation_0-rmse:0.01773
[783]	validation_0-rmse:0.01772
[784]	validation_0-rmse:0.01771
[785]	validation_0-rmse:0.01770
[786]	validation_0-rmse:0.01770
[787]	validation_0-rmse:0.01769
[788]	validation_0-rmse:0.01768
[789]	validation_0-rmse:0.01767
[790]	validation_0-rmse:0.01767
[791]	validation_0-rmse:0.01766
[792]	validation_0-rmse:0.01764
[793]	validation_0-rmse:0.01764
[794]	validation_0-rmse:0.01763
[795]	validation_0-rmse:0.01762
[796]	validation_0-rmse:0.01760
[797]	validation_0-rmse:0.01759
[798]	validation_0-rmse:0.01759
[799]	validation_0-rmse:0.01758
[800]	validation_0-rmse:0.01757
[801]	validation_0-rmse:0.01756
[802]	validation_0-rmse:0.01754
[803]	validation_0-rmse:0.01753
[804]	va

### Predictions

In [9]:
predictions = model.predict(test_features)
predictions[:5]

array([-0.9188895 , -0.85928416, -0.8576763 , -0.92040676, -0.9286955 ],
      dtype=float32)

In [10]:
actual_close = df[['Close']]
actual_close = pd.DataFrame(actual_close.iloc[train_len:, 0])

In [11]:
predictions = pd.DataFrame(predictions)
predictions.reset_index(drop=True, inplace=True)
predictions.index = test_features.index
predictions['Actual'] = actual_close['Close']
predictions.rename(columns={0:'Pred'}, inplace=True)

In [12]:
predictions

Pred    Actual
Date                          
2020-09-10 -0.918890 -0.943262
2020-09-11 -0.859284 -0.882979
2020-09-14 -0.857676 -0.879433
2020-09-15 -0.920407 -0.911348
2020-09-16 -0.928695 -0.911348
...              ...       ...
2022-09-12  0.849026  0.872340
2022-09-13  0.864243  0.907801
2022-09-14  0.799584  0.845745
2022-09-15  0.820817  0.863475
2022-09-16  0.767403  0.819149

[492 rows x 2 columns]

In [13]:
predictions = predictions.dropna()

In [14]:
predictions

Pred    Actual
Date                          
2020-09-10 -0.918890 -0.943262
2020-09-11 -0.859284 -0.882979
2020-09-14 -0.857676 -0.879433
2020-09-15 -0.920407 -0.911348
2020-09-16 -0.928695 -0.911348
...              ...       ...
2022-09-12  0.849026  0.872340
2022-09-13  0.864243  0.907801
2022-09-14  0.799584  0.845745
2022-09-15  0.820817  0.863475
2022-09-16  0.767403  0.819149

[492 rows x 2 columns]

In [15]:
predictions['Actual'].plot(figsize=(20,8), legend=True, color='blue')
predictions['Pred'].plot(legend=True, color='red', figsize=(20,8))

<AxesSubplot:xlabel='Date'>

### Evaluation

**Inverse transform scaled Close price**

In [16]:
from joblib import load

In [17]:
close_scaler = load('../feature_engineering/isat_close_scaler.bin')

In [18]:
inversed_pred = close_scaler.inverse_transform(np.array(predictions['Pred']).reshape(-1,1))
inversed_act = close_scaler.inverse_transform(np.array(predictions['Actual']).reshape(-1,1))

In [19]:
inversed = pd.DataFrame(inversed_pred)
inversed['Actual'] = inversed_act
inversed.rename({0:'Pred'}, axis=1, inplace=True)
inversed.index = test_labels.index
inversed.head(2)

Pred  Actual
Date                           
2020-09-10  2148.731445  2080.0
2020-09-11  2316.818604  2250.0

**Plot error distribution**

In [20]:
def plotErrorHist(inversed_df):
    """
    Function to plot error histogram
    """
    error = inversed_df['Pred'] - inversed_df['Actual']
    plt.figure(figsize=(10,6))
    plt.hist(error, bins=25)
    plt.xlabel('Prediction Error [Close]')
    _ = plt.ylabel('Count')

In [21]:
plotErrorHist(inversed)

**RMSE**

In [22]:
from statsmodels.tools.eval_measures import rmse

In [23]:
rmse(inversed['Pred'], inversed['Actual'])

205.05238314700364

**MAPE**

In [24]:
# mape
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [25]:
mape(inversed['Actual'], inversed['Pred'])

2.218765437314547

**R-squared**

In [26]:
from sklearn.metrics import r2_score

In [27]:
r2_score(inversed['Actual'], inversed['Pred'])

0.9814151967436648

### Save Model

In [28]:
# Save model
model.save_model('./models/isat.json')